In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

#### Set Embeddings

In [4]:
emb = OpenAIEmbeddings()

#### Docs to index

In [5]:
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

#### Load

In [6]:
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

#### Split

In [7]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=500, chunk_overlap=0)
doc_splits = text_splitter.split_documents(docs_list)

#### Add to vectorstore

In [8]:
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=emb,
)
retriever = vectorstore.as_retriever()

## LLMs

In [9]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

#### Data Model

In [10]:
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["vectorstore", "web_search"] = Field(
        ...,
        description="Given a user question, choose to route it to web search or a vectorstore.",
    )

#### LLM with function call

In [11]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm_router = llm.with_structured_output(RouteQuery)

#### Prompt

In [12]:
system = """You are an expert at routing a user question to a vectore or web search.
The vectorstore contains documents related to agents, prompt engineering, and adversarial attacks.
Use the vectorstore for questions on these topics. Otherwise, use web-search."""
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | structured_llm_router
print(
    question_router.invoke(
        {"question": "Who will the bears draft first in the NFL draft?"}
    )
)
print(question_router.invoke({"question": "What are the types of agent memory?"}))

datasource='web_search'
datasource='vectorstore'


## Retrieval Grader

#### Data Model

In [13]:
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

#### LLM with function call

In [14]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

#### Prompt

In [16]:
system = """You are a grader assessing relevance of a retrieved document to a user question. \n
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader
question = "agent memory"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

binary_score='yes'


## Generate

In [17]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

#### Prompt

In [18]:
prompt = hub.pull("rlm/rag-prompt")

/home/longview/anaconda3/envs/langgraph/lib/python3.12/site-packages/langsmith/client.py:322: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/home/longview/anaconda3/envs/langgraph/lib/python3.12/site-packages/langsmith/client.py:5301: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


#### LLM

In [19]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

#### Post-processing

In [20]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

#### Chain

In [21]:
rag_chain = prompt | llm | StrOutputParser()

#### Run

In [22]:
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

Agent memory in LLM-powered autonomous agents refers to a long-term memory module that records the agent's experiences in natural language. This memory allows agents to reflect on past events, synthesize higher-level inferences, and inform future behavior based on relevance, recency, and importance. It enhances the agent's ability to interact and make decisions in a dynamic environment.


## Hallucination Grader

#### Data Model

In [23]:
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )

#### LLM with function call

In [24]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeHallucinations)

#### Prompt

In [25]:
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n
    Give a binary 'yes' or 'no' score. 'Yes' means that the answer is grounded in / supported by the set of facts."""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)

hallucinations_grader = hallucination_prompt | structured_llm_grader
hallucinations_grader.invoke({"documents": docs, "generation": generation})

GradeHallucinations(binary_score='yes')

## Answer Grader

#### Data Model

In [26]:
class GradeAnswer(BaseModel):
    """Binary score to assess if answer addresses the question."""

    binary_score: str = Field(
        description="Answer addressess the question, 'yes' or 'no'"
    )

#### LLM with function call

In [27]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
structured_llm_acc_grader = llm.with_structured_output(GradeAnswer)

#### Prompt

In [28]:
system = """You are a grader assessing whether an answer addresses / resolves a question \n
    Give a binary 'yes' or 'no' score. 'Yes' means that the answer resolves the question."""
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}")
    ]
)

answer_grader = answer_prompt | structured_llm_grader
answer_grader.invoke({"question": question, "generation": generation})

GradeHallucinations(binary_score='yes')

## Question Re-writer

#### LLM

In [29]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

#### Prompt

In [31]:
system = """You are a question re-writer that converts an input question into a better version that is optimized \n
    for vectorstore retrieval. Look at the input and try to reason about the underlying semantic intent / meaning."""
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Here is the initial question: \n\n {question} \n Formulate an improved question.")
    ]
)

question_rewriter = re_write_prompt | llm
question_rewriter.invoke({"question": question})

AIMessage(content='What are the key concepts and techniques related to agent memory in artificial intelligence?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 69, 'total_tokens': 84, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'stop', 'logprobs': None}, id='run-abdf1b16-1e27-4f9c-a270-3d18c4aaaede-0', usage_metadata={'input_tokens': 69, 'output_tokens': 15, 'total_tokens': 84})